## Classification Model

This file details the classification model we used to predict the likelihood of Alzheimer's in female patients.

In [4]:
# Necessary Installations
pip install numpy pandas tensorflow scikit-learn
pip install git+https://github.com/genentech/gReLU.git

We import the needed libraries and files  below:

In [6]:
import pandas as pd
import grelu.resources
import grelu.sequence.format
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm  # Import tqdm for progress bar
import numpy as np

/usr/local/lib/python3.10/dist-packages/enformer_pytorch/modeling_enformer.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  TF_GAMMAS = torch.load(str(DIR / "precomputed

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The data is then loaded and labelled accordingly:

In [8]:
# Load your CSV file without headers, treating all lines as sequences

data = pd.read_csv('/content/drive/My Drive/BioInformatics Hackathon/output.csv', header=None)  # No headers in the CSV file
data.columns = ['sequence']  # Assign a header to the column

# Ensure even and odd rows are labeled as "good" or "faulty"
data['label'] = ['good' if i % 2 == 0 else 'faulty' for i in range(len(data))]

# Now, you have the sequences with labels
sequences = data['sequence']

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vheahe-work (vheahe-work-university-of-toronto). Use `wandb login --relogin` to force relogin
wandb: Downloading large artifact human_fold0:latest, 711.00MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:7.9


Model is loaded from gReLU:

In [ ]:
# Load the Borzoi model from gReLU
model = grelu.resources.load_model(
    project="borzoi",
    model_name="human_fold0"
)


Processing and data manipulation is performed before a Random Forest classifier is trained to predict the relationship between specific genetic mutations and the probability of a patient having Alzheimer's.

In [ ]:
# Convert sequences into a format that Borzoi can understand
# Make sure all sequences are padded to match the expected input length
expected_length = model.data_params['train_seq_len']

def pad_sequence(seq, length):
    return seq + 'A' * (length - len(seq)) if len(seq) < length else seq[:length]

# Add a progress bar for padding sequences
input_seqs = [pad_sequence(seq, expected_length) for seq in tqdm(sequences, desc="Padding sequences")]

# Use Borzoi to make predictions on the input sequences
print("Running Borzoi model predictions...")

# Add a progress bar for predictions
predictions = []
for seq in tqdm(input_seqs, desc="Predicting sequences"):
    pred = model.predict_on_seqs([seq], device="cpu")  # Predict one sequence at a time to show progress
    predictions.append(pred)

# Convert predictions list to numpy array
predictions = np.vstack(predictions)

# Borzoi outputs predictions (e.g., probability tracks); use these as features
# Flatten the predictions for Random Forest input
predictions_flat = predictions.reshape(predictions.shape[0], -1)

# Prepare the features (X) and labels (y) for Random Forest
X = pd.DataFrame(predictions_flat)  # Borzoi predictions as features
y = data['label']  # Labels: 'good' or 'faulty'

# Encode the labels to numerical values: 0 for 'faulty', 1 for 'good'
y = y.map({'faulty': 0, 'good': 1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # Define the classifier
clf.fit(X_train, y_train)  # Train the classifier

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

# Save the Random Forest model for future use
joblib.dump(clf, 'random_forest_alzheimer_model.pkl')

Padding sequences: 100%|██████████| 7116/7116 [00:05<00:00, 1359.17it/s]


Running Borzoi model predictions...


Predicting sequences:   0%|          | 0/7116 [00:00<?, ?it/s]